# Рекомендация тарифов

Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта «Определение перспективного тарифа для телефонной компании»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — она уже сделана.

Необходимо построить модель с максимально большим значением *accuracy*, но не меньше 0.75. Проверить *accuracy* на тестовой выборке.


## 1. Откройте и изучите файл

In [13]:
# импортируем все необходимые библиотеки
# pandas
import pandas as pd

# модель логистической регрессии, но она нам пригодится, потому что у нас задача классификации, а не прогноза конкретного значения
from sklearn.linear_model import LogisticRegression  

# модель дерева решений
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor

# модель случайного леса
from sklearn.ensemble import RandomForestClassifier

# метод для разделения выборки на тренировочную и валидационную
from sklearn.model_selection import train_test_split

# метод для расчета mse
from sklearn.metrics import mean_squared_error

# метод для расчета accuracy
from sklearn.metrics import accuracy_score

# библиотека для выгрузки модели в файл
import joblib

In [9]:
# посмотрим на исходные данные
#df = pd.read_csv('https://code.s3.yandex.net/datasets/users_behavior.csv')
#df = pd.read_csv('https://code.s3.yandex.net/datasets/users_behavior.csv')
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


Отлично, исходные данные уже преведены к задаче классификации. Если в поле "is_ultra" стоит 0, то означает что тариф ***не*** Ultra, а значит что он Smart, если стоит 1, то тариф Ultra.

**Вывод**

Исходные данные приведены к задачи классификации. 
Мы имеем выбоку клиентов, которые уже перешли на один из двух тарифов и должны научиться оперелять по косвенным признакам, какой из тарифов выбрал абонент. В дальнейшем мы хотим применить эту модель, чтобы предлагать старым клиентам один из двух тарифов наиболее подходящий, взамен архивного. 

## 2. Разбейте данные на выборки

In [10]:
# разобъем данные на тренировочную, тестовую и валидационную выборки

df_train, df_valid = train_test_split(df, test_size=0.2, random_state=12345)

# сначала выделим тестовую выборку, которая должна быть 20%
df_train, df_test = train_test_split(df, test_size=0.2, random_state=12345)

# теперь оставшиеся 80% разобъем на тренировочную и валидационную выборку, 
# так, чтоб тренировочная была 60% от датасета, а валидационная 20% от датасета
# другими словами, 75% и 25% от тех 80% что остались

df_train, df_valid = train_test_split(df_train, test_size=0.25, random_state=12345)


# обозначим зависимые и независимые переменные для тренировочной, тестовой и валидационной выборок

features_train = df_train.drop('is_ultra',axis=1)
target_train = df_train['is_ultra']

features_test = df_test.drop('is_ultra',axis=1)
target_test = df_test['is_ultra']

features_valid = df_valid.drop('is_ultra',axis=1)
target_valid = df_valid['is_ultra']



## 3. Исследуйте модели

Будем использовать модель дерева решений, модель случайного леса и модель логистической регрессии. В свою очередь, у модели случайного леса могут быть различные гиперпараметры, такие как глубина (max_depth) и количество деревьев (n_estimators).

Чтобы не плодить однотипный код, оформим в функицию процесс обучения модели и расчета ее accuracy. Функция будет получать на вход модель, тестовые и валидационные фоеймы и будет возвращать accuracy тренировочной и валидационной выборок.

In [11]:
def models(model, features_train, features_valid, target_train, target_valid):
    # обучим модель
    model.fit(features_train,target_train)

    # построим предсказания модели
    predictions_train = model.predict(features_train)
    predictions_valid = model.predict(features_valid)
    

    # посмотрим, насколько точна модель
    
    return accuracy_score(target_train, predictions_train), accuracy_score(target_valid, predictions_valid)

Отлично, функцию написали. Теперь посмотрим на модель дерева решений

In [14]:
# объявим модель дерева решений
model_tree = DecisionTreeRegressor(random_state=12345)

# и вызовем функцию в которой обучим модель и посмотрим accuracy 
accuracy_train, accuracy_valid = models(model_tree, features_train, features_valid, target_train, target_valid)

print('Accuracy модели')
print('train:',accuracy_train)
print('valid:',accuracy_valid)

Accuracy модели
train: 1.0
valid: 0.7122861586314152


Не очень то и хорошо. Дерево решений отлично определяет где какой тариф на тренировочной выборке (еще-бы, вызубрил!), но вот на 
валидационной выборке точность такая себе - всего 71%. Не очень результат.

Посмотрим на модели случайного леса с разными гиперпарраметрами глубины (max_depth) и разным количеством деревьев (n_estimators)

In [18]:
# создадим словарь, в котором ключами будут значения глубины дерева и количества деревьев в лесу,
# а значениями - accuracy валидационной выборки
results={}

# в цикле переберем различные комбинации глубины дерева и количества деревьев и запишем соответствующие accaracy валидационной выборки в словарь
for depth in range(1,20,1):
    for estim in range(1,20,1):
        # объявим модель
        model_forest = RandomForestClassifier(random_state=12345, max_depth=depth, n_estimators=estim, min_samples_split=5)
        
        # воспользуемся ранее описанной функцией
        accuracy_train, accuracy_valid = models(model_forest, features_train, features_valid, target_train, target_valid)
        
        # создадим ключ для словаря
        keys = 'depth='+str(depth)+", estim="+str(estim)
        
        # создадим значение для ключа
        val = accuracy_valid 
        
        # запишем пару "ключ: значение" в словарь
        results.update({str(keys):val})


In [19]:
# посмотрим при каком ключе у нас получилось максимальное значение 
key_max_accuracy = max(results, key=results.get)

print("Максимальная accuracy модели случайного леса достигается при",key_max_accuracy)
print("и точность модели на валидационных данных составляет {:.2%}".format(results.get(key_max_accuracy)))

Максимальная accuracy модели случайного леса достигается при depth=14, estim=3
и точность модели на валидационных данных составляет 80.72%


Чтож, уже лучше. При лесе из 3х деревьев с глубиной дерева не более 14 уровней точность предсказаний составляет уже 80%. Обучим модель случайного леса на этих параметрах и в дальнейшем проверим модель на тестовой выборке.

In [20]:
model_forest = RandomForestClassifier(random_state=12345, max_depth=14, n_estimators=3, min_samples_split=5)
model_forest.fit(features_train,target_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=14, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=5,
                       min_weight_fraction_leaf=0.0, n_estimators=3,
                       n_jobs=None, oob_score=False, random_state=12345,
                       verbose=0, warm_start=False)

Модель случайного леса мы обучили на тестовой выборке и подобрали ей параметры на валидационной выборке. Теперь ее можно отложить, к ней мы вернемся позже, чтобы посмотреть на ее точность на тестовой выборке (спойлер: 78%). А пока посмотрим, имеет ли смысл использовать модель логистической регрессии.

In [21]:
model_logistic_regression = LogisticRegression()
model_logistic_regression.fit(features_train,target_train)
predictions_valid_logistic_regression = model_logistic_regression.predict(features_valid)
print(accuracy_score(target_valid,predictions_valid_logistic_regression))

0.6967340590979783


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Как видим, на валидационных данных модель логисчтической регрессии показывает самую малую точность, всео 69.7%. 

*N.B.: получили сообщение, что в следующей версии библиотеки sklearn нам нужно будет указывать параметр max_iter в явном виде, при использовании логистической регрессии*

**Вывод**

Среди моделей логистической регрессии, дерева решений и случайного леса мы выбрали модель, которая илучшим образом классифицирует данные на тренировочной и валидационной выборках. Это модель случайного леса и для нее мы подобрали такие параметры глубины деревьев и количества деревьев, при которых достигается максимальная точность на валидационной выборке.

Теперь самое время проверить настроенную нами модель случайного леса на тестовой выборке и узнать, на сколько она хороша.

## 4. Проверьте модель на тестовой выборке

Ранее мы выбрали, настроили и обучили модель случайного леса. Нам остается лишь сделать предсказания и посмотреть на их точность. Для этого используем тестовую выборку.

In [147]:
predictions_test_forest = model_forest.predict(features_test)
print("Точность модели случайного леса на тестовой выборке составляет {:.2%}".format(accuracy_score(target_test, predictions_test_forest)))

Точность модели случайного леса на тестовой выборке составляет 78.23%


Воу! Не так уж и ужастно. Точность на тестовой выборке лучше чем на тренировочной выборке у модели дерева решений и модели логистической регрессии.

## 5. (бонус) Проверьте модели на адекватность

Проверка адекватности модели будет заключаться в проверке качества модели в сравнении с константной моделью. Если модель плохо предсказывает заранее известные нам константы, то модель будем считать неадекватной.
Предскажем для всего теста метку наибольшего класса, посчитаем accuracy и сравним с accuracy нашей модели.

In [148]:
# посмотрим количество значений в каждом из классов

users_Smart=df[df['is_ultra']==0]['is_ultra'].count()
users_Ultra = df[df['is_ultra']==1]['is_ultra'].count()

print("Всего абонентов на тарифе Ultra:",total_Ultra)
print("Всего абонентов на тарифе Smart:",total_Smart)


Всего абонентов на тарифе Ultra: 985
Всего абонентов на тарифе Smart: 2229


Отлично, значит проверку адекватности модели будем производить на выборке из пользователей тарифа Smart, это и есть метка наибольшего класса

In [149]:
# определим зависимые и независимые переменные
features_sanity_check = df.query('is_ultra==0').drop('is_ultra',axis=1)
target_sanity_check = df.query('is_ultra==0')['is_ultra']

# предскажем значения зависимых переменных
predictions_sanity_check = model_forest.predict(features_sanity_check)

# посчитаем и выведем accuracy
accuracy_sanity_check = accuracy_score(target_sanity_check, predictions_sanity_check)
print("Точность модели случайного леса в проверке на адекватность {:.2%}".format(accuracy_sanity_check))

Точность модели случайного леса в проверке на адекватность 94.12%


**вывод**

Выбранная модель достаточно точно предсказывает значения и может считаться адекватной.

## Общий вывод

Моделью для задачи классификация является модель случайного леса с деревьями не глубже 14 уровней и количество деревьев в лесу - 3 штуки. Эта модель адекватна и предсказывает с точностью 78.23% на тестовой выборке. Возможно, она могла бы и точнее (поработав с параметрами глубины и количества деревьев), но в рамках проекта нам достаточно точности и в 75%. Модель выгружена в файл 'model.joblib'.

In [151]:
joblib.dump(model_forest,'model.joblib')

['model.joblib']